In [125]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/testfile/Test.xlsx
/kaggle/input/best39705/Train.xlsx


In [126]:
import pandas as pd
import numpy as np

In [127]:
df = pd.read_excel('/kaggle/input/best39705/Train.xlsx',header=0)
df.head()

,NO,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q69,Q70,Q71,Q72,Q73,Q74,Q75,Q76,Q77,CLASS
0,11,1,5,2,5,1,3,1,1,1,...,4,3,1,4,1,2,2,3,4,1
1,12,2,5,1,5,1,5,1,1,1,...,5,3,1,5,1,2,1,3,1,1
2,13,2,4,2,5,2,5,2,2,2,...,4,3,2,3,2,3,2,2,4,1
3,14,2,5,1,5,2,5,2,1,2,...,3,2,2,4,2,2,4,3,4,1
4,15,1,4,1,5,1,3,1,1,1,...,4,2,2,3,1,3,1,3,5,1


In [128]:
df.shape

(281, 79)

In [129]:
df_clean = df.dropna(0,'any')

In [130]:
df_clean.shape

(280, 79)

In [131]:
pel_class = np.unique(df['CLASS'])
pel_class

array([1, 2, 3])

In [132]:
pel_value = [sum(df_clean['CLASS']==pel_class[0]),
             sum(df_clean['CLASS']==pel_class[1]),
             sum(df_clean['CLASS']==pel_class[2])]
pel_value

[134, 85, 61]

In [133]:
y = df_clean['CLASS'].values
X = df_clean.drop(['CLASS'],axis = 1).values

In [134]:
np.shape(y)

(280,)

In [135]:
np.shape(X)

(280, 78)

In [136]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X,y)

In [137]:
sum(y_res==3)

134

## SVM + FEATURE SELECTION 

In [138]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res,test_size=0.15,random_state=0)

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
fs = SelectKBest(chi2, k=3)
fs.fit(X_train, y_train)
X_train = fs.transform(X_train)
X_test = fs.transform(X_test)

#from sklearn.decomposition import PCA
#fr = PCA(n_components=3)
#fr.fit(X_train)
#X_train = fr.transform(X_train)
#X_test = fr.transform(X_test)


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
params = {
    'C' : [1,2,4], # High C = Overfitting
    'gamma' : [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32] # High gamma = Overfitting
}
clf = GridSearchCV(SVC(),params, cv=10)
clf.fit(X_train_norm, y_train)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))

Best params : {'C': 4, 'gamma': 1}
10CV accuracy : 98.82352941176471


In [139]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_predict = clf.predict(X_test_norm)
print("Test accuracy : "+str(sum(y_test == y_predict)/len(y_test)*100))

Test accuracy : 95.08196721311475


## Test

In [140]:
df = pd.read_excel('/kaggle/input/testfile/Test.xlsx',header=0)


In [141]:
yt = df['CLASS'].values
Xt = df.drop(['CLASS'],axis = 1).values

In [142]:
X_test = fs.transform(Xt)

In [143]:
X_test_norm = scaler.transform(X_test)

In [ ]:
y_predict = clf.predict(X_test_norm)
print("Test accuracy : "+str(sum(yt == y_predict)/len(yt)*100))